<a href="https://colab.research.google.com/github/PETERMWAURA254/Boston-Housing-Price-Prediction/blob/main/hate_speech_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn spacy transformers imbalanced-learn
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 5.1 MB/s eta 0:00:00


 ### **Import** **libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
# dataset shape to know how many tweets in the datasets
print(f"num of tweets: {df.shape}")

# extract the text and labels
tweet = list(df['tweet'])
labels = list(df['class'])

num of tweets: (24783, 6)


 ### **Preprocess**  **dataset**

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Remove URLs, special characters, and extra spaces
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase
    # Tokenization and Lemmatization using spaCy
nlp = spacy.load('en_core_web_sm')
# Preprocessing function
def preprocess_text(text):
    # Remove URLs, special characters, and extra spaces
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase

# Tokenization and Lemmatization using spaCy
import spacy
nlp = spacy.load('en_core_web_sm')
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])
  # Apply preprocessing
df['cleaned_tweet'] = df['tweet'].apply(preprocess_text)
df['lemmatized_tweet'] = df['cleaned_tweet'].apply(lemmatize_text)
df['lemmatized_tweet'] = df['cleaned_tweet'].apply(lemmatize_text)

In [ ]:
print(df.columns)

Index(['count', 'hate_speech_count', 'offensive_language_count',
       'neither_count', 'class', 'tweet', 'cleaned_tweet', 'lemmatized_tweet'],
      dtype='object')


### Prepare dataset for model training

In [ ]:
X = df['lemmatized_tweet']
y = df['class']  # Assuming label column indicates hate speech (0: non-hate, 1: hate)

###  Split data into training and testing *sets*

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Handling Imbalanced Data with SMOTE

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text into numerical features using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)  # Convert training text to TF-IDF vectors
X_test_tfidf = tfidf.transform(X_test)  # Convert test text to TF-IDF vectors

# Handle Class Imbalance using SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

### Logistic Regression model
    

In [ ]:
# Logistic Regression Model Training
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_smote, y_train_smote)

LogisticRegression(max_iter=1000)

In [ ]:
# Predictions
y_pred_logreg = logreg.predict(X_test_tfidf)

In [ ]:
# Evaluation
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.57      0.38       290
           1       0.96      0.85      0.90      3832
           2       0.78      0.90      0.84       835

    accuracy                           0.85      4957
   macro avg       0.67      0.78      0.71      4957
weighted avg       0.89      0.85      0.86      4957



### Load BERT

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Load the dataset
df = pd.read_csv("train.csv")

# Prepare features and labels
X = df['tweet']
y = df['class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the TinyBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_4L_312D')

# Custom dataset class for PyTorch
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(list(texts), padding='max_length', truncation=True,
                                  max_length=max_length, return_tensors="pt")
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Create datasets for training and testing
train_dataset = HateSpeechDataset(X_train, y_train, tokenizer)
test_dataset = HateSpeechDataset(X_test, y_test, tokenizer)

# Load TinyBERT model with the correct number of labels
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_4L_312D',
                                                          num_labels=len(y.unique()))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Define training arguments with optimizations
training_args = TrainingArguments(
    output_dir='./tinybert_results',
    num_train_epochs=3,                  # Increased epochs for better performance
    per_device_train_batch_size=32,       # Optimized batch size for TinyBERT
    per_device_eval_batch_size=64,        # Larger eval batch size
    eval_strategy="steps",              # Evaluate during training
    eval_steps=500,                       # Evaluate every 500 steps
    logging_dir='./logs',
    logging_steps=250,                    # Log every 250 steps
    save_strategy="epoch",               # Save model at each epoch
    fp16=torch.cuda.is_available(),       # Use mixed precision if GPU available
    gradient_accumulation_steps=2,        # Helps with memory efficiency
    warmup_steps=100,                     # More warmup steps for better stability
    weight_decay=0.01,                    # Regularization to reduce overfitting
    load_best_model_at_end=True,          # Load best model when training ends
    metric_for_best_model="eval_loss",   # Use evaluation loss to select best model
    greater_is_better=False,              # Lower loss is better
    dataloader_num_workers=4              # Increase worker threads for faster loading
)

# Define compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': (preds == labels).mean(),
        'precision': precision,
        'recall': recall,
        'f1-score': f1
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training...")
trainer.train()
print("Training complete!")

# Evaluate the model
print("Evaluating model...")
trainer.evaluate()

# Generate predictions
predictions = trainer.predict(test_dataset)
y_pred_bert = np.argmax(predictions.predictions, axis=1)

# Print Classification Report
print("TinyBERT Classification Report:")
print(classification_report(y_test, y_pred_bert))

# Save the model
model_path = './hate_speech_tinybert_model'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

# Example of how to use the model for inference
def predict_hate_speech(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()

    return prediction, probs

# Test with some examples
test_texts = [
    "I really enjoyed this product",
    "You are such an idiot",
    "I hate everyone from that country"
]

print("\nTesting model with example texts:")
for text in test_texts:
    pred, probs = predict_hate_speech(text)
    print(f"Text: '{text}'")
    print(f"Prediction: {pred}")
    print(f"Probabilities: {probs.cpu().numpy()[0]}")
    print("---")


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Using device: cpu


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...
{'loss': 0.579, 'grad_norm': 2.3132224082946777, 'learning_rate': 5e-05, 'epoch': 0.3225806451612903}
